In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 利用頂點 AI 上的生成式模型進行文字摘要

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/doggy8088/generative-ai/blob/main/language/prompts/examples/text_summarization.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory 標誌"><br>在 Colab 中執行
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/doggy8088/generative-ai/blob/main/language/prompts/examples/text_summarization.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub 標誌"><br>在 GitHub 上查看
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/blob/main/language/prompts/examples/text_summarization.zh.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="頂點 AI 標誌"><br>在頂點 AI 工作台中開啟
    </a>
  </td>
</table>


| | |
|-|-|
|作者 | [Polong Lin](https://github.com/polong-lin) |


## 概觀
文字摘要產生一份較長文字文件簡潔且流利的摘要。有兩種主要的文字摘要類型：抽取式和抽象式。抽取式摘要包含選擇原始文字中的關鍵句子，並將它們組合起來形成摘要。抽象式摘要包含產生新句子，代表原始文字中的重點。在本筆記中，你將深入了解幾個範例，說明大型語言模型如何協助根據文字產生摘要。

在 [官方文件](https://cloud.google.com/vertex-ai/docs/generative-ai/text/summarization-prompts) 中進一步瞭解文字摘要。


### 目標

在本教學中，你將學習如何使用生成模型，透過以下範例摘要文字中的資訊：
- 腳本摘要
- 將文字摘要成要點
- 包含待辦事項的對話摘要
- 標籤代幣化
- 標題和標題產生

你還將學習如何透過使用 ROUGE 作為評估框架，將模型生成的摘要與人工製作的摘要進行比較，進而評估模型。


### 費用

本教學指南使用 Google Cloud 的計費元件：

* Vertex AI 生成式 AI Studio

了解 [Vertex AI 價格](https://cloud.google.com/vertex-ai/pricing)，
並使用 [價格計算器](https://cloud.google.com/products/calculator/)
根據預計使用情況產生費用估算值。


## 開始使用


### 安裝 Vertex AI SDK


In [ ]:
!pip install google-cloud-aiplatform --upgrade --user

**僅 Colab：** 取消下一個Cell註解以重新啟動Kernel或使用按鈕重新啟動Kernel。對於 Vertex AI Workbench，你可以使用頂端的按鈕重新啟動終端機。


In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### 驗證筆記本環境
* 如果你使用 **Colab** 執行此筆記本，取消註解下方的Cell並繼續。
* 如果你使用 **Vertex AI 工作台** ，請查看[此處](https://github.com/doggy8088/generative-ai/tree/main/setup-env)的設定說明。


In [ ]:
# from google.colab import auth
# auth.authenticate_user()

### 匯入函式庫

讓我們從匯入本教學課程所需的函式庫開始


**僅限 Colab：** 取消下方單元格的註解，以初始化 Vertex AI SDK。對於 Vertex AI Workbench，不需要執行此動作。


In [ ]:
# import vertexai

# PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
# vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
from vertexai.language_models import TextGenerationModel

### 匯入模型

在此，我們載入名為 `text-bison@001` 的預訓練文字生成模型。


In [ ]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

## 文本摘要


### 逐字稿摘要

在第一個範例中，你針對量子運算的文字摘要一段。


In [ ]:
prompt = """
Provide a very short summary, no more than three sentences, for the following article:

Our quantum computers work by manipulating qubits in an orchestrated fashion that we call quantum algorithms.
The challenge is that qubits are so sensitive that even stray light can cause calculation errors — and the problem worsens as quantum computers grow.
This has significant consequences, since the best quantum algorithms that we know for running useful applications require the error rates of our qubits to be far lower than we have today.
To bridge this gap, we will need quantum error correction.
Quantum error correction protects information by encoding it across multiple physical qubits to form a “logical qubit,” and is believed to be the only way to produce a large-scale quantum computer with error rates low enough for useful calculations.
Instead of computing on the individual qubits themselves, we will then compute on logical qubits. By encoding larger numbers of physical qubits on our quantum processor into one logical qubit, we hope to reduce the error rates to enable useful quantum algorithms.

Summary:

"""

print(
    generation_model.predict(
        prompt, temperature=0.2, max_output_tokens=1024, top_k=40, top_p=0.8
    ).text
)

取代摘要，我們可以要求 TL；DR (「太長；沒讀」)。你可以比較輸出之間產生的區別。


In [ ]:
prompt = """
Provide a TL;DR for the following article:

Our quantum computers work by manipulating qubits in an orchestrated fashion that we call quantum algorithms. 
The challenge is that qubits are so sensitive that even stray light can cause calculation errors — and the problem worsens as quantum computers grow. 
This has significant consequences, since the best quantum algorithms that we know for running useful applications require the error rates of our qubits to be far lower than we have today. 
To bridge this gap, we will need quantum error correction. 
Quantum error correction protects information by encoding it across multiple physical qubits to form a “logical qubit,” and is believed to be the only way to produce a large-scale quantum computer with error rates low enough for useful calculations. 
Instead of computing on the individual qubits themselves, we will then compute on logical qubits. By encoding larger numbers of physical qubits on our quantum processor into one logical qubit, we hope to reduce the error rates to enable useful quantum algorithms.

TL;DR:
"""

print(
    generation_model.predict(
        prompt, temperature=0.2, max_output_tokens=1024, top_k=40, top_p=0.8
    ).text
)

### 將文字總結成要點
在下例中，你於量子計算上使用相同文字，但要求模型以要點形式將其總結出來。歡迎你變更提示。


In [ ]:
prompt = """
Provide a very short summary in four bullet points for the following article:

Our quantum computers work by manipulating qubits in an orchestrated fashion that we call quantum algorithms.
The challenge is that qubits are so sensitive that even stray light can cause calculation errors — and the problem worsens as quantum computers grow.
This has significant consequences, since the best quantum algorithms that we know for running useful applications require the error rates of our qubits to be far lower than we have today.
To bridge this gap, we will need quantum error correction.
Quantum error correction protects information by encoding it across multiple physical qubits to form a “logical qubit,” and is believed to be the only way to produce a large-scale quantum computer with error rates low enough for useful calculations.
Instead of computing on the individual qubits themselves, we will then compute on logical qubits. By encoding larger numbers of physical qubits on our quantum processor into one logical qubit, we hope to reduce the error rates to enable useful quantum algorithms.

Bulletpoints:

"""

print(
    generation_model.predict(
        prompt, temperature=0.2, max_output_tokens=256, top_k=1, top_p=0.8
    ).text
)

### 與待辦事項的對話摘要
對話摘要涉及將對話濃縮成較短的格式，以便你不需要閱讀整個討論，但可以利用摘要。在此範例中，你要求模型整理在線上零售客戶與支援代理之間的範例對話，並在最後加入待辦事項。


In [ ]:
prompt = """
Please generate a summary of the following conversation and at the end summarize the to-do's for the support Agent:

Customer: Hi, I'm Larry, and I received the wrong item.

Support Agent: Hi, Larry. How would you like to see this resolved?

Customer: That's alright. I want to return the item and get a refund, please.

Support Agent: Of course. I can process the refund for you now. Can I have your order number, please?

Customer: It's [ORDER NUMBER].

Support Agent: Thank you. I've processed the refund, and you will receive your money back within 14 days.

Customer: Thank you very much.

Support Agent: You're welcome, Larry. Have a good day!

Summary:
"""

print(
    generation_model.predict(
        prompt, temperature=0.2, max_output_tokens=256, top_k=40, top_p=0.8
    ).text
)

###  Hashtag 分詞
Hashtag 分詞是處理一段文字並取得其 hashtag「Token」的過程。例如，如果你想要為你的社群媒體活動產生 hashtag，你就可以使用這個方法。在這個範例中，你可以使用 [這篇 Google Cloud 的推文](https://twitter.com/googlecloud/status/1649127992348606469) 並產生一些可用的 hashtag。


In [ ]:
prompt = """
Tokenize the hashtags of this tweet:

Google Cloud
@googlecloud
How can data help our changing planet? 🌎

In honor of #EarthDay this weekend, we’re proud to share how we’re partnering with
@ClimateEngine
 to harness the power of geospatial data and drive toward a more sustainable future.

Check out how → https://goo.gle/3mOUfts
"""

print(
    generation_model.predict(
        prompt, temperature=0.8, max_output_tokens=1024, top_k=40, top_p=0.8
    ).text
)

### 標題和標題產生
在下面，你要求模型為特定文本產生五個可能的標題／標題組合選項。


In [ ]:
prompt = """
Write a title for this text, give me five options:
Whether helping physicians identify disease or finding photos of “hugs,” AI is behind a lot of the work we do at Google. And at our Arts & Culture Lab in Paris, we’ve been experimenting with how AI can be used for the benefit of culture.
Today, we’re sharing our latest experiments—prototypes that build on seven years of work in partnership the 1,500 cultural institutions around the world.
Each of these experimental applications runs AI algorithms in the background to let you unearth cultural connections hidden in archives—and even find artworks that match your home decor."
"""

print(
    generation_model.predict(
        prompt, temperature=0.8, max_output_tokens=256, top_k=1, top_p=0.8
    ).text
)

## 評估
你可以使用 [ROUGE](https://en.wikipedia.org/wiki/ROUGE_(metric)) 作為評估架構來評估摘要任務的輸出。ROUGE (面向召回的精要評估替身) 是一種自動確定摘要品質的測量方式，方法是將其與人類建立的其他 (理想的) 摘要做比較。這些測量方法計算重疊單元數，例如 n 元組、字詞順序，和電腦自動產生的摘要與人類建立的理想摘要之間的字詞對。


第一步是安裝 ROUGE 函式庫。


In [ ]:
!pip install rouge

從一個語言模型建立摘要，你可以使用它來與人類生成的摘要比較。


In [ ]:
from rouge import Rouge

ROUGE = Rouge()

prompt = """
Provide a very short, maximum four sentences, summary for the following article:

Our quantum computers work by manipulating qubits in an orchestrated fashion that we call quantum algorithms.
The challenge is that qubits are so sensitive that even stray light can cause calculation errors — and the problem worsens as quantum computers grow.
This has significant consequences, since the best quantum algorithms that we know for running useful applications require the error rates of our qubits to be far lower than we have today.
To bridge this gap, we will need quantum error correction.
Quantum error correction protects information by encoding it across multiple physical qubits to form a “logical qubit,” and is believed to be the only way to produce a large-scale quantum computer with error rates low enough for useful calculations.
Instead of computing on the individual qubits themselves, we will then compute on logical qubits. By encoding larger numbers of physical qubits on our quantum processor into one logical qubit, we hope to reduce the error rates to enable useful quantum algorithms.

Summary:

"""

candidate = generation_model.predict(
    prompt, temperature=0.1, max_output_tokens=1024, top_k=40, top_p=0.9
).text

print(candidate)

你還需要一位人為產生的摘要，我們將利用這個摘要來與模型所產生的「候選者」進行比較。我們將稱此為「參照」。


In [ ]:
reference = "Quantum computers are sensitive to noise and errors. To bridge this gap, we will need quantum error correction. Quantum error correction protects information by encoding across multiple physical qubits to form a “logical qubit”."

現在你可以取得候選項目和參考以評估效能。在此案例中，ROUGE 會提供給你：

- `rouge-1`，用於衡量單字共現頻率
- `rouge-2`，用於衡量二字共現頻率
- `rouge-l`，用於衡量最長公共子序列


In [ ]:
ROUGE.get_scores(candidate, reference)